# Normalization Challenge

In [1]:
# Importing packages
import numpy as np
from numpy import argmax
import pandas as pd

# Keras
from keras.models import Sequential
from keras.layers import Input, Flatten
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.preprocessing.text import text_to_word_sequence
from keras.layers import Embedding

Using TensorFlow backend.


In [7]:
df = pd.read_csv("en_train.csv")
# Transforming the data into vectors of size 50 each
df['length_after'] = df['after'].astype(str).map(len)
df['length_before'] = df['before'].astype(str).map(len)
df = df[(df['length_after'] <= 30) & (df['length_before'] <= 30)]
max_num_features = 30
x_data = []
y_data = []

for x in df['before'].values[:1000000]:
    x_row = np.zeros(max_num_features, dtype=int)
    #  Taking the ASCII value of an expression
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    x_data.append(x_row)
    
for y in df['after'].values[:1000000]:
    y_row = np.zeros(max_num_features, dtype=int)
    #  Taking the ASCII value of an expression
    for yi, i in zip(list(str(y)), np.arange(max_num_features)):
        y_row[i] = ord(yi)
    y_data.append(y_row)

In [8]:
x_data = np.array(x_data)
y_data = np.array(y_data)

y_data = y_data[:,:,np.newaxis] # Adding the axis

x_data = x_data[:1000000]
y_data = y_data[:1000000]

In [9]:
# Calculating the maximum value of the dataset
np.max(x_data)

65364

In [23]:
vocab = np.max(x_data)
model = Sequential()
model.add(Embedding(vocab, 100, embeddings_initializer='uniform', input_length = 30, trainable=True))
model.add(LSTM(50, return_sequences=True))
#model.add(RepeatVector(50))
#model.add(LSTM(128, return_sequences=True))

#model.add(Dense(50, activation='softmax'))
model.add(TimeDistributed(Dense(vocab + 1, activation='softmax')))

#model.add(Dense(vocab, activation='softmax'))
model.compile(loss=custom_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
print(model.summary())

history = model.fit(x_data, y_data, epochs=1, verbose=1,validation_split=0.2)

AttributeError: 'Tensor' object has no attribute 'tolist'

In [22]:
# Custom loss function with costs
import keras.backend as K
from itertools import product

def custom_categorical_crossentropy(y_pred, y_true):
    is_plain = True
    for item in ['.', '%', '@', ' ', '!', '1','2','3']:
        if (item in y_pred):
            is_plain = False
    if (is_plain):
        loss = K.sparse_categorical_crossentropy(y_pred, y_true)
    else:
        loss = K.sparse_categorical_crossentropy(y_pred, y_true) * 10
    return loss